**Add ViT code**

In [6]:
import torch
from transformers import ViTModel, ViTFeatureExtractor
from PIL import Image

# Load pretrained ViT model
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# Load and preprocess the image
# image = Image.open("/home/ds/Research/lcd/orbslam3_docker/Datasets/KITTI/data_odometry_gray/dataset/sequences/00/image_0/000000.png")
image = Image.open("checkboard_7.jpg")
inputs = feature_extractor(images=image, return_tensors="pt")

# Extract features
with torch.no_grad():
    outputs = model(**inputs)

# Extract the CLS token as the place descriptor
place_descriptor = outputs.last_hidden_state[:, 0, :]  # Shape: (1, 768)
print(place_descriptor.shape)  # (1, 768)


/home/ds/miniconda3/envs/netvlad/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


torch.Size([1, 768])


In [7]:
place_descriptor

tensor([[ 0.0481, -0.2065,  0.0631,  0.0353,  0.1081, -0.2221,  0.0036,  0.1943,
         -0.0445, -0.3849, -0.1312,  0.3608,  0.2188,  0.2202, -0.2179, -0.1992,
          0.0409, -0.3344, -0.0285, -0.1340,  0.1087, -0.3145, -0.1056,  0.2532,
          0.0587, -0.2087, -0.1091,  0.0106, -0.1033,  0.0574,  0.1291,  0.3898,
          0.1081,  0.2517, -0.0446, -0.3196, -0.0538,  0.2293,  0.2498, -0.2004,
          0.2402,  0.0850,  0.0926, -0.3626, -0.1459,  0.1840, -0.0821, -0.4395,
          0.0420,  0.0928, -0.0443,  0.0791,  0.1611, -0.2228, -0.0389,  0.0117,
         -0.1011,  0.0681,  0.1490,  0.0162,  0.0156, -0.0220,  0.5471, -0.1246,
          0.0195, -0.1269,  0.2465, -0.1130, -0.3081, -0.4494,  0.2540, -0.0963,
         -0.1244,  0.0351,  0.0513,  0.1245, -0.0142, -0.1594, -0.0404,  0.1563,
          0.0183, -0.1800,  0.0819,  0.2815, -0.2191,  0.1200,  0.1453,  0.0298,
         -0.0651, -0.1544, -0.0502,  0.2575,  0.2558, -0.1156, -0.0203, -0.2655,
          0.3792, -0.2572,  

In [5]:
outputs.last_hidden_state.shape

torch.Size([1, 197, 768])

In [8]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load image
image = Image.open("checkboard_7.jpg")

# Resize and convert to tensor
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
image_tensor = transform(image).unsqueeze(0)  # Add batch dim

# Convert to patches (assuming 16x16 patch size)
patch_size = 16
patches = image_tensor.unfold(2, patch_size, patch_size).unfold(3, patch_size, patch_size)
patches = patches.contiguous().view(1, 3, -1, patch_size, patch_size)  # Reshape to (B, C, N, P, P)

print("Patches shape:", patches.shape)  # (1, 3, 196, 16, 16) for 224x224 image


Patches shape: torch.Size([1, 3, 196, 16, 16])


In [18]:
inputs

{'pixel_values': tensor([[[[ 0.4196,  0.3020,  0.2314,  ..., -0.5373, -0.5373, -0.5373],
          [ 0.4196,  0.3098,  0.2471,  ..., -0.5373, -0.5294, -0.5294],
          [ 0.4353,  0.3255,  0.2549,  ..., -0.5294, -0.5294, -0.5294],
          ...,
          [ 0.7804,  0.6314,  0.4353,  ..., -0.5529, -0.5529, -0.5608],
          [ 0.7882,  0.6392,  0.4275,  ..., -0.5608, -0.5529, -0.5608],
          [ 0.7961,  0.6471,  0.4196,  ..., -0.5608, -0.5608, -0.5608]],

         [[ 0.4275,  0.3098,  0.2235,  ..., -0.5451, -0.5451, -0.5451],
          [ 0.4353,  0.3176,  0.2392,  ..., -0.5451, -0.5373, -0.5373],
          [ 0.4510,  0.3255,  0.2471,  ..., -0.5373, -0.5373, -0.5373],
          ...,
          [ 0.7412,  0.5922,  0.3961,  ..., -0.6314, -0.6314, -0.6314],
          [ 0.7490,  0.6078,  0.3882,  ..., -0.6314, -0.6314, -0.6314],
          [ 0.7647,  0.6157,  0.3804,  ..., -0.6314, -0.6314, -0.6314]],

         [[ 0.4118,  0.2784,  0.1922,  ..., -0.7020, -0.7020, -0.7020],
          [ 0

In [17]:
patches

tensor([[[[[0.7098, 0.6510, 0.6157,  ..., 0.5647, 0.5765, 0.5922],
           [0.7098, 0.6549, 0.6235,  ..., 0.5647, 0.5804, 0.5922],
           [0.7176, 0.6627, 0.6275,  ..., 0.5608, 0.5765, 0.5922],
           ...,
           [0.7686, 0.6941, 0.6392,  ..., 0.5529, 0.5647, 0.5804],
           [0.7686, 0.6980, 0.6471,  ..., 0.5490, 0.5647, 0.5804],
           [0.7765, 0.6980, 0.6471,  ..., 0.5490, 0.5647, 0.5725]],

          [[0.6118, 0.6353, 0.6510,  ..., 0.7882, 0.7922, 0.7922],
           [0.6078, 0.6314, 0.6510,  ..., 0.7882, 0.7922, 0.7922],
           [0.6078, 0.6275, 0.6510,  ..., 0.7882, 0.7922, 0.7922],
           ...,
           [0.5961, 0.6118, 0.6353,  ..., 0.8000, 0.8000, 0.8078],
           [0.5922, 0.6078, 0.6353,  ..., 0.7961, 0.8000, 0.8000],
           [0.5882, 0.6078, 0.6353,  ..., 0.8000, 0.8039, 0.8000]],

          [[0.7922, 0.7961, 0.7922,  ..., 0.7647, 0.7608, 0.7608],
           [0.7922, 0.8000, 0.7961,  ..., 0.7647, 0.7608, 0.7608],
           [0.7961, 0.8000

TypeError: ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0): ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivati